In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
v0 = 0.5172
r = 0
T = 1 / 12
K = 1000
xi = 0.6
# symbolic variables
s, v, t = symbols('s v t')
sigma = sqrt(v0)
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / (sigma * sqrt(t))
D0 = 0.5 * (v - v0) * s * density(N)(d_p) / (sigma * sqrt(t))
delta0 = diff(D0, s)
gamma0 = diff(delta0, s)
vega0 = diff(D0, v)

S = [950+10*i for i in range(11)]

def delta(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return N(d_p)

def vega(S, K, T, r, sigma):
  N = norm.pdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  return S * N(d_p) * np.sqrt(T) / (2 * sigma)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

D = [D0]
for _ in range(1, 5):
  D.append(L(D[-1]))

In [2]:
Delta = [delta0]
Delta_func = []
for i in range(1, 5):
  Delta.append(diff(D[i], s))
for i in range(0, 5):
  Delta_func.append(lambdify([s, v, t], Delta[i]))

Vega = [vega0]
Vega_func = []
for i in range(1, 5):
  Vega.append(diff(D[i], v))
for i in range(0, 5):
  Vega_func.append(lambdify([s, v, t], Vega[i]))

In [3]:
%%time
Approx_delta = []
for s in S:
  delta0_approx = delta(s, K, T, r, np.sqrt(v0)) + T * Delta_func[0](s, v0, T)
  delta1_approx = delta0_approx + T**2 / 2 * Delta_func[1](s, v0, T)
  delta2_approx = delta1_approx + T**3 / 6 * Delta_func[2](s, v0, T)
  delta3_approx = delta2_approx + T**4 / 24 * Delta_func[3](s, v0, T)
  delta4_approx = delta3_approx + T**5 / 120 * Delta_func[4](s, v0, T)
  Approx_delta.append(100*delta4_approx)
print(Approx_delta)

[44.28744596171008, 46.29785263392905, 48.29678302410081, 50.279384404884, 52.24107440513339, 54.17756201002294, 56.08486406457631, 57.959317443826016, 59.7975871051441, 61.59667028068839, 63.35389710075023]
Wall time: 5.71 s


In [4]:
%%time
Approx_vega = []
for s in S:
  vega0_approx = vega(s, K, T, r, np.sqrt(v0))
  vega1_approx = vega0_approx + T**2 / 2 * Vega_func[1](s, v0, T)
  vega2_approx = vega1_approx + T**3 / 6 * Vega_func[2](s, v0, T)
  vega3_approx = vega2_approx + T**4 / 24 * Vega_func[3](s, v0, T)
  vega4_approx = vega3_approx + T**5 / 120 * Vega_func[4](s, v0, T)
  Approx_vega.append(vega4_approx)
print(Approx_vega)

[74.89173042816458, 76.1418254966514, 77.20296874419397, 78.07309332286293, 78.75164447564721, 79.23951102322653, 79.53894282105415, 79.6534565714064, 79.58773236188433, 79.34750324088877, 78.93944003908216]
Wall time: 4.87 s


In [5]:
Delta_MC = np.array([43.5844, 45.5741, 47.6381, 49.8072, 51.6445, 53.8656, 55.9119, 58.0716, 60.0880, 62.0285, 63.6870])
Delta_diff = (Approx_delta - Delta_MC) / Delta_MC * 100
Delta_JFE = np.array([44.2419, 46.2644, 48.2756, 50.2706, 52.2445, 54.1930, 56.1119, 58.9976, 59.8464, 61.6554, 63.4218])
Delta_JFE_diff = np.array([1.5085, 1.5147, 1.3383, 0.9304, 1.1618, 0.6265, 0.3578, -0.1276, -0.4020, -0.6015, -0.4163])

Vega_MC = np.array([74.2766, 75.5218, 76.5901, 77.4442, 78.1237, 78.5703, 78.8212, 78.8205, 78.6555, 78.3327, 77.9749])
Vega_diff = (Approx_vega - Vega_MC) / Vega_MC * 100
Vega_JFE = np.array([74.9072, 76.1723, 77.2457, 78.1252, 78.8099, 79.3008, 79.6002, 79.7115, 79.6396, 79.3905, 78.9711])
Vega_JFE_diff = np.array([0.8489, 0.8614, 0.8561, 0.8793, 0.8784, 0.9298, 0.9883, 1.1305, 1.2512, 1.3505, 1.2776])

df = pd.DataFrame({
    "S": S, 
    "MC %delta": Delta_MC, 
    "Replicate %delta": np.round(Approx_delta, 4), 
    "Replicate delta %diff": np.round(Delta_diff, 4),
    "JFE %delta": Delta_JFE,
    "JFE delta %diff": Delta_JFE_diff,
    "MC %vega": Vega_MC,
    "Replicate %vega": np.round(Approx_vega, 4),
    "Replicate vega %diff": np.round(Vega_diff, 4),
    "JFE %vega": Vega_JFE,
    "JFE vega %diff": Vega_JFE_diff
    })
print(df)
df.to_csv("Panel AB1 (v=0.5172).csv")

       S  MC %delta  Replicate %delta  Replicate delta %diff  JFE %delta  \
0    950    43.5844           44.2874                 1.6131     44.2419   
1    960    45.5741           46.2979                 1.5881     46.2644   
2    970    47.6381           48.2968                 1.3827     48.2756   
3    980    49.8072           50.2794                 0.9480     50.2706   
4    990    51.6445           52.2411                 1.1552     52.2445   
5   1000    53.8656           54.1776                 0.5791     54.1930   
6   1010    55.9119           56.0849                 0.3094     56.1119   
7   1020    58.0716           57.9593                -0.1934     58.9976   
8   1030    60.0880           59.7976                -0.4833     59.8464   
9   1040    62.0285           61.5967                -0.6962     61.6554   
10  1050    63.6870           63.3539                -0.5230     63.4218   

    JFE delta %diff  MC %vega  Replicate %vega  Replicate vega %diff  \
0            1.